In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col

spark = SparkSession.builder.getOrCreate()

# Step 1: Load Bronze Data
df_raw = spark.read.format("delta").load("/mnt/delta/bronze/web_form_submissions")

# Step 2: Clean & Transform
df_clean = df_raw.select(
    "submission_id",
    "full_name",
    "email",
    "phone",
    "address",
    "comments",
    to_timestamp("submitted_at").alias("submitted_at"),
    "source_file"
).dropna(subset=["submission_id", "submitted_at"])

# Step 3: Write Silver
df_clean.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/delta/silver/web_forms_clean")

# Optional Table Registration
spark.sql("DROP TABLE IF EXISTS web_forms_clean")
spark.sql("""
    CREATE TABLE web_forms_clean
    USING DELTA
    LOCATION '/mnt/delta/silver/web_forms_clean'
""")
